In [ ]:
!pip -q install spacy                                                            # instalacja biblioteki
!python -m spacy download pl_core_news_sm -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 69.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pl_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import spacy

# zaladowanie modelu jezyka polskiego
nlp = spacy.load("pl_core_news_sm")

#wczytanie danych
df = pd.read_csv("dane1.csv")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'dane1.csv'

In [ ]:
# wybranie pierwszego pustego tweeta
sample_text = df["text"].dropna().astype(str).iloc[0]
sample_text

In [ ]:
# rozpoznanie encji w jednym tweecie
doc = nlp(sample_text)
[(ent.text, ent.label_) for ent in doc.ents]

In [ ]:
# wyciagniecie osob, miejsc i organizacji
def extract_entities(text):
  if not isinstance(text, str) or not text.strip():
    return [], [], []

  doc = nlp(text)                                             # analiza tekstu przez model NLP

  persons = []
  places = []
  orgs = []

  # etykiety spotykane w modelach NER
  person_labels   = {"persName", "PERSON", "PER"}
  place_labels    = {"placeName", "LOC", "GPE"}
  org_labels      = {"orgName", "ORG"}


  # sprawdzamy każdą wykrytą encję
  for ent in doc.ents:
    if ent.label_ in person_labels:
      persons.append(ent.text.strip())
    elif ent.label_ in place_labels:
      places.append(ent.text.strip())
    elif ent.label_ in org_labels:
      orgs.append(ent.text.strip())

  #usunecie duplikatow
  def dedupe(seq):
    seen = set()
    out = []
    for x in seq:
      if x not in seen:
        out.append(x)
        seen.add(x)
    return out

  return dedupe(persons), dedupe(places), dedupe(orgs)

In [ ]:
persons_list = []
places_list = []
orgs_list = []

# przejscie po wszystkich tweetach
for t in df["text"].fillna("").astype(str):
  p, pl, o = extract_entities(t)
  persons_list.append(p)
  places_list.append(pl)
  orgs_list.append(o)

# dodanie nowych kolumn
df["persons"]          = persons_list
df["places"]           = places_list
df["organizations"]    = orgs_list

df[["text", "persons", "places", "organizations"]].head(10)

In [ ]:
df.to_csv("dane1_part3_ner.csv", index = False)

from google.colab import files
files.download("dane1_part3_ner.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>